# Notebook 01: Exploratory Data Analysis - NEMT Rides

**Author:** Hector Carbajal  
**Version:** 1.1  
**Last Updated:** 2026-02

---

## Purpose
This notebook performs initial data exploration on Non-Emergency Medical Transport (NEMT) ride data. We aim to baseline the current operational performance and identify data quality issues before calculating efficiency scores.

## Inputs
- `data/raw/trips.csv` - Raw trip records
- `data/raw/drivers.csv` - Driver information

## Key Stakeholder Questions
- What is the current on-time pickup rate across the fleet?
- Are there specific regions or trip types driving operational late-starts?
- What is our average vehicle capacity utilization?

## Table of Contents
1. [Setup & Environment Check](#setup)
2. [Data Audit & Outlier Detection](#audit)
3. [Temporal Load Distribution](#temporal)
4. [Geographic Distribution](#geographic)
5. [Operational Performance Baseline](#performance)
6. [💡 Key Findings](#findings)

In [1]:
# Setup
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
import sys
from scipy import stats

# Add project root to path
sys.path.insert(0, str(Path.cwd().parent))
from src.config import RAW_DIR, PROCESSED_DIR
from src.data_generation import generate_trips, generate_drivers, save_raw_data

# Global Aesthetic Config
px.defaults.template = "plotly_white"
COLOR_PRIMARY = "#1f77b4"

print("✅ Setup complete")

✅ Setup complete


<a id="audit"></a>
## 1. Data Audit & Outlier Detection
*Identifying statistical anomalies in trip telemetry.*

In [2]:
# Load or generate data
trips_file = RAW_DIR / "trips.csv"
if not trips_file.exists():
    trips_df = generate_trips()
    save_raw_data(trips_df, "trips.csv")
else:
    trips_df = pd.read_csv(trips_file, parse_dates=[
        "requested_pickup_time", "scheduled_pickup_time", 
        "actual_pickup_time", "actual_dropoff_time"
    ])

def audit_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return len(outliers), lower_bound, upper_bound

print("📋 ANOMALY DETECTION:")
for col in ['distance_miles']:
    count, low, high = audit_outliers(trips_df, col)
    print(f"- {col:15} | Outliers: {count:4} | Expected Range: [{low:.1f}, {high:.1f}]")

📋 ANOMALY DETECTION:
- distance_miles  | Outliers:    0 | Expected Range: [-0.4, 10.2]


<a id="temporal"></a>
## 2. Temporal Load Analysis
*Identifying 'Rush Hour' for NEMT dispatching.*

In [3]:
trips_df['hour'] = trips_df['scheduled_pickup_time'].dt.hour
hourly_vol = trips_df.groupby('hour').size().reset_index(name='trips')

fig = px.line(
    hourly_vol, x='hour', y='trips', 
    title='Intra-Day Trip Volume (Pickup Demand)',
    labels={'hour': 'Hour of Day (24h)', 'trips': 'Volume'},
    markers=True
)
fig.show()

<a id="findings"></a>
## 💡 Key Findings
**Operational Readiness Summary**

In [4]:
on_time_rate = 1 - trips_df['late_pickup_flag'].fillna(0).astype(int).mean()
avg_utilization = (trips_df['num_passengers'] / trips_df['vehicle_capacity']).mean()

print("="*80)
print("🚀 NEMT OPS BASELINE")
print("="*80)
print(f"1. RELIABILITY: Fleet-wide On-Time Pickup rate is {on_time_rate:.1%}.")
print(f"2. CAPACITY: Vehicle utilization is averaging {avg_utilization:.1%}. Potential for shared-ride routing detected.")
print(f"3. ANOMALIES: Detected {trips_df['distance_miles'].quantile(0.99):.1f}+ mile trips which require secondary logic validation.")
print("="*80)

🚀 NEMT OPS BASELINE
1. RELIABILITY: Fleet-wide On-Time Pickup rate is 92.0%.
2. CAPACITY: Vehicle utilization is averaging 64.6%. Potential for shared-ride routing detected.
3. ANOMALIES: Detected 8.4+ mile trips which require secondary logic validation.
